In [3]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup

from tabula import read_pdf
from tabulate import tabulate
from itertools import chain
# If encounter SSL certify error - go to your Python 3.x.x folder and install Certification 

In [15]:
data = pd.read_csv('farm_smoke.csv')
sonoma_data = data.query('County == "Sonoma"').copy().reset_index(drop=True)
sonoma_data

,County,Year,Links
0,Sonoma,2021,https://sonomacounty.ca.gov//Main%20County%20S...
1,Sonoma,2020,https://sonomacounty.ca.gov//Main%20County%20S...
2,Sonoma,2019,https://sonomacounty.ca.gov//Main%20County%20S...
3,Sonoma,2018,https://sonomacounty.ca.gov//Main%20County%20S...
4,Sonoma,2017,https://sonomacounty.ca.gov//Main%20County%20S...
...,...,...,...
89,Sonoma,1932,https://sonomacounty.ca.gov//Main%20County%20S...
90,Sonoma,1931,https://sonomacounty.ca.gov//Main%20County%20S...
91,Sonoma,1930,https://sonomacounty.ca.gov//Main%20County%20S...
92,Sonoma,1929,https://sonomacounty.ca.gov//Main%20County%20S...


In [16]:
url = sonoma_data['Links'][0]
url

'https://sonomacounty.ca.gov//Main%20County%20Site/Natural%20Resources/Agricultural%2C%20Weights%20%26%20Measures/Documents/Crop%20Reports/2021-Sonoma-County-Crop-Report.pdf'

In [6]:
# to get these info, use Adobe or Mac's preview 
# Preview: Tools --> show inspector 
# follow this documentation: https://github.com/chezou/tabula-py/issues/13

top = 158.9
left = 36.32
height = 431.61
width = 540.13

y1 = top
x1 = left
y2 = top + height
x2 = left + width

In [8]:
sonoma_2013_red = read_pdf(url, pages=12, area=[y1, x1, y2, x2], silent=True, stream=True, multiple_tables=False)[0]
sonoma_2013_red.fillna(method='ffill', inplace=True)
sonoma_2013_red

KeyboardInterrupt: 

In [26]:
for index, row in sonoma_2021_red.iterrows():
    if len(str(row['Year'])) > 4:
        print(row['Bearing'].split())

['242.0', '244', '0']
['278.0', '279', '0']


In [33]:
for index, row in sonoma_2021_red.iterrows():
    print(row['Year'])

2021
2020
2021
2020
2021
2020
2021
2020
2021
2020
2021
2020
2021
2020
2021
2020
2021 2020
2021
2020
2021 2020
2021
2020
2021
2020
2021
2020


In [20]:
import re

def clean_dataframe(df):
    new_rows = []
    for index, row in df.iterrows():
        if len(str(row['Year'])) > 4:
            varietal = row['Varietal']
            bearing, non_bearing = row['Bearing'].split()
            total_bearing, total_non_bearing = row['Total'].split()
            tons_bearing, tons_non_bearing = row['Tons'].split()
            price_bearing, price_non_bearing = row['$/Ton'].split()
            value_bearing, value_non_bearing = row['Total Value'].split('$')
            years = re.split('\s+', row['Year'])
            new_rows.append([varietal, years[0], bearing, non_bearing, total_bearing, 
                             tons_bearing, price_bearing, value_bearing])
            new_rows.append([varietal, years[1], bearing, non_bearing, total_non_bearing, 
                             tons_non_bearing, price_non_bearing, value_non_bearing])
        else:
            new_rows.append(row)
            
    new_df = pd.DataFrame(new_rows, columns=df.columns)
    new_df['Bearing'] = new_df['Bearing'].astype(float)
    new_df['Non-Bearing'] = new_df['Non-Bearing'].astype(float)
    new_df['Total'] = new_df['Total'].astype(float)
    new_df['Tons'] = new_df['Tons'].astype(float)
    new_df['$/Ton'] = new_df['$/Ton'].apply(lambda x: float(x.strip('$').replace(',', '')))
    new_df['Total Value'] = new_df['Total Value'].apply(lambda x: float(x.strip('$').replace(',', '')))
    return new_df

# df = pd.read_csv('your_file.csv')
cleaned_df = clean_dataframe(sonoma_2021_red)
print(cleaned_df)

ValueError: too many values to unpack (expected 2)

In [175]:
# sonoma_2021_white = read_pdf(url, pages=13, area=[y1, x1, y2, x2], silent=True, stream=True, guess=False)[0]
# sonoma_2021_white.fillna(method='ffill', inplace=True)
# sonoma_2021_white

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Chardonnay,2021,"15,049.0",451.0,"15,500.0","66,833.0","$2,284.72","$152,694,700"
1,Chardonnay,2020,"15,030 0",482 0,"15,512 0","47,877 7","$2,311 46","$110,667,400"
2,Gewürztraminer,2021,99.0,0.0,99.0,197.7,"$1,997.98","$395,000"
3,Gewürztraminer,2020,99 0,0 0,99 0,138 5,"$2,098 65","$290,700"
4,Muscat Blanc/,2021,17.0,0.0,17.0,93.4,"$2,339.75","$218,500"
5,Muscat Canelli,2020,17 0,0 0,17 0,82 5,"$2,544 76","$209,900"
6,Pinot Blanc,2021 2020,84.0 84 0,0.0 0 0,84.0 84 0,183.5 132 9,"$2,114.26 $2,352 48","$388,000$312,600"
7,Pinot Gris,2021 2020,397.0 408 0,0.0 0 0,397.0 408 0,"1,157.6 1,040 6","$1,865.39 $1,841 05","$2,159,400$1,915,800"
8,Roussanne,2021,36.0,3.0,39.0,75.4,"$3,509.65","$264,600"
9,Roussanne,2020,36 0,0 0,36 0,63 2,"$2,496 12","$157,800"


In [ ]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

lens = sonoma_2021_red['Bearing'].str.split(' ').map(len)

In [166]:
sonoma_2021_red['Bearing'].str.split(' ').map(len)

0     1
1     2
2     1
3     2
4     1
5     2
6     1
7     2
8     1
9     2
10    1
11    2
12    1
13    2
14    1
15    2
16    3
17    1
18    2
19    3
20    1
21    2
22    1
23    2
24    1
25    2
Name: Bearing, dtype: int64

In [6]:
# sonoma_2021_red['Bearing'] = sonoma_2021_red['Bearing'].str.replace(' ', '.')
# sonoma_2021_red

In [7]:
# cols_to_convert = ['Bearing', 'Non-Bearing', 'Total', 'Tons'] 
# for col in cols_to_convert:
#     sonoma_2021_red[col] = sonoma_2021_red[col].str.replace(',', '').astype(float).round(1)

# sonoma_2021_red

In [8]:
# pd.to_numeric(sonoma_2021_red['Bearing'].str.replace(' ', '.'))

In [9]:
# Check if a row has two values in the 'Year' column
has_two_years = sonoma_2021_red['Year'].str.contains('\s+')

In [10]:
# sonoma_2021_red.set_index(['Varietal', 'Year', 'Bearing'])\
#     .apply(lambda x: x.str.split(' ').explode()).reset_index()

In [11]:
# sonoma_2021_red.iloc[[16]].explode()

In [12]:
# has_two_years = sonoma_2021_red['Year'].str.contains('\s+')
# split_rows = []

In [9]:
data = pd.read_csv('farm_smoke.csv')
sonoma_data = data.query('County == "Sonoma"').copy().reset_index(drop=True)
sonoma_data

,County,Year,Links
0,Sonoma,2021,https://sonomacounty.ca.gov//Main%20County%20S...
1,Sonoma,2020,https://sonomacounty.ca.gov//Main%20County%20S...
2,Sonoma,2019,https://sonomacounty.ca.gov//Main%20County%20S...
3,Sonoma,2018,https://sonomacounty.ca.gov//Main%20County%20S...
4,Sonoma,2017,https://sonomacounty.ca.gov//Main%20County%20S...
...,...,...,...
89,Sonoma,1932,https://sonomacounty.ca.gov//Main%20County%20S...
90,Sonoma,1931,https://sonomacounty.ca.gov//Main%20County%20S...
91,Sonoma,1930,https://sonomacounty.ca.gov//Main%20County%20S...
92,Sonoma,1929,https://sonomacounty.ca.gov//Main%20County%20S...


In [10]:
url = sonoma_data['Links'][8]
url

'https://sonomacounty.ca.gov//Main%20County%20Site/Natural%20Resources/Agricultural%2C%20Weights%20%26%20Measures/Documents/Crop%20Reports/2013_Crop%20Report.pdf'

In [11]:
top = 93
left = 40.45
height = 437.69
width = 538.27

y1 = top
x1 = left
y2 = top + height
x2 = left + width

In [12]:
sonoma_2013_red = read_pdf(url, pages=8, area=[y1, x1, y2, x2], silent=True, stream=True, multiple_tables=False)[0]
# sonoma_2013_red.fillna(method='ffill', inplace=True)
sonoma_2013_red

,Red Varieties Year,Bearing,Non-Bearing Total,Tons,$/Ton,Unnamed: 5,Total Value
0,2013,604.5,13.2 617.7,"1,917.8","$2,445.23",NaN,"$4,689,500"
1,Cabernet Franc,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,661.3,2.7 664.0,"1,906.5","$2,325.46",NaN,"$4,433,500"
3,2013,"11,830.3","1,070.8 12,901.1","44,255.8","$2,462.99",NaN,"$109,001,600"
4,CabeTronents Soafu Gvirganpoens Crus2h0e1d2,"11,929.4","1W68i.0n e1 2G,0r97a.4p e 4P7,1r9o4.d0 ucti$o2...",NaN,NaN,NaN,"$109,186,200"
5,2013,579.6,4.0 5A8c3r.e6s,506.7,"$1,870.18",NaN,"Production $947,700"
6,Carignane,NaN,NaN,NaN,NaN,NaN,NaN
7,Top 12 R2e0d1 V2a rieties 612.Y4e ar,NaN,Beari0n.g0 Non6-B1e2a.r4i ng,51T0o.6ta l,"$1,T8o7n9s.74",NaN,"$/Ton $9T5o9t,a8l 0V0a l"
8,Tons\t\r of\t\r Grapes\t\r Crushed,NaN,NaN,NaN,NaN,NaN,NaN
9,Cabe2rn0e1t F3r anc,425.270 12,160692..34 535.1 2.4,"2,706.666 4.7","$21,,930566.5.19 $",NaN,"2,325.46 $$6,377,43,40303,"
